In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[4]').appName('DF DataSkew Optimization').getOrCreate()
spark

23/06/30 22:36:25 WARN Utils: Your hostname, OMEN resolves to a loopback address: 127.0.1.1; using 172.19.181.52 instead (on interface eth0)
23/06/30 22:36:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/30 22:36:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Sample Dataset
### Sale dataset:
#### **Table1**: OrderId, Qty, Sales, Discount (yes=1, no=0)
#### **Table2**: ProductId, OrderId, Product, Price

In [3]:
import numpy as np
import pandas as pd
import random

In [4]:
# Table 1:
# (key, key_count, quantity, sales)
config = (101,100,100,(10,100)),(201,7000000,200,(50,3400)),(301,500,1000,(12,2000)),(401,10000,50,(40,1000))
OrderID = []
Sales = []
Qty = []

for key, key_count, quantity, sales in config:
    OrderID.extend([key] * key_count)
    Qty.extend(list(np.random.randint(low=1, high=quantity, size=key_count)))
    Sales.extend(list(np.random.randint(low=sales[0], high=sales[1], size=key_count)))
random.shuffle(OrderID)
Discount = list(np.random.randint(low=0, high=2, size=len(OrderID)))
data1 = list(zip(OrderID, Qty, Sales, Discount))

data_skew = pd.DataFrame(data1, columns=['OrderID', 'Qty', 'Sales', 'Discount'])

# Table 2:
data2 = [
    [1, 101, 'Pencil', 4.99],
    [2, 101, 'book', 9.5],
    [3, 101, 'scissors', 14],
    [4, 301, 'glue', 7],
    [5, 201, 'marker', 8.49],
    [6, 301, 'label', 2],
    [7, 201, 'calculator', 3.99],
    [8, 501, 'eraser', 1.55]
]

data_small = pd.DataFrame(data2, columns=['ProductID', 'OrderID', 'Product', 'Price'])

In [11]:
# Optimize conversion between PySpark and Pandas dataframes
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', 'true')

df_skew = spark.createDataFrame(data_skew)
df_skew = df_skew.coalesce(4)
df_skew.show()
df_skew.rdd.getNumPartitions()

+-------+---+-----+--------+
|OrderID|Qty|Sales|Discount|
+-------+---+-----+--------+
|    201| 70|   75|       0|
|    201| 49|   28|       0|
|    201| 32|   79|       1|
|    201| 42|   69|       1|
|    201| 44|   39|       0|
|    201| 51|   56|       1|
|    201| 15|   47|       0|
|    201| 95|   35|       1|
|    201| 20|   73|       1|
|    201|  5|   13|       0|
|    201| 83|   16|       1|
|    201| 65|   72|       1|
|    201| 90|   46|       0|
|    201|  9|   88|       0|
|    201| 12|   18|       0|
|    201| 85|   95|       0|
|    201| 61|   91|       1|
|    201| 69|   38|       1|
|    201| 51|   30|       0|
|    201| 43|   87|       1|
+-------+---+-----+--------+
only showing top 20 rows



23/06/30 22:46:58 WARN TaskSetManager: Stage 8 contains a task of very large size (54752 KiB). The maximum recommended task size is 1000 KiB.


4

In [12]:
df_small = spark.createDataFrame(data_small)
df_small.printSchema()
df_small.show()
df_small.rdd.getNumPartitions()

root
 |-- ProductID: long (nullable = true)
 |-- OrderID: long (nullable = true)
 |-- Product: string (nullable = true)
 |-- Price: double (nullable = true)

+---------+-------+----------+-----+
|ProductID|OrderID|   Product|Price|
+---------+-------+----------+-----+
|        1|    101|    Pencil| 4.99|
|        2|    101|      book|  9.5|
|        3|    101|  scissors| 14.0|
|        4|    301|      glue|  7.0|
|        5|    201|    marker| 8.49|
|        6|    301|     label|  2.0|
|        7|    201|calculator| 3.99|
|        8|    501|    eraser| 1.55|
+---------+-------+----------+-----+



4

In [13]:
joined_df = df_skew.join(df_small, on='OrderID', how='inner')

In [14]:
joined_df.show(30)

+-------+---+-----+--------+---------+----------+-----+
|OrderID|Qty|Sales|Discount|ProductID|   Product|Price|
+-------+---+-----+--------+---------+----------+-----+
|    201| 70|   75|       0|        7|calculator| 3.99|
|    201| 70|   75|       0|        5|    marker| 8.49|
|    201| 49|   28|       0|        7|calculator| 3.99|
|    201| 49|   28|       0|        5|    marker| 8.49|
|    201| 32|   79|       1|        7|calculator| 3.99|
|    201| 32|   79|       1|        5|    marker| 8.49|
|    201| 42|   69|       1|        7|calculator| 3.99|
|    201| 42|   69|       1|        5|    marker| 8.49|
|    201| 44|   39|       0|        7|calculator| 3.99|
|    201| 44|   39|       0|        5|    marker| 8.49|
|    201| 51|   56|       1|        7|calculator| 3.99|
|    201| 51|   56|       1|        5|    marker| 8.49|
|    201| 15|   47|       0|        7|calculator| 3.99|
|    201| 15|   47|       0|        5|    marker| 8.49|
|    201| 95|   35|       1|        7|calculator

23/06/30 22:48:34 WARN TaskSetManager: Stage 10 contains a task of very large size (54752 KiB). The maximum recommended task size is 1000 KiB.


In [15]:
joined_df.count()

23/06/30 22:48:39 WARN TaskSetManager: Stage 12 contains a task of very large size (54752 KiB). The maximum recommended task size is 1000 KiB.


14001300

In [16]:
joined_df.rdd.getNumPartitions()

4

In [17]:
spark.conf.get('spark.sql.shuffle.partitions')

'200'

In [18]:
spark.conf.set('spark.sql.shuffle.partitions', 8)

In [20]:
from pyspark.sql.functions import *

In [21]:
summary = joined_df.select(
    mean(joined_df.Sales).alias('AVG(Sales)'),
    stddev(joined_df.Sales).alias('STD(Sales)'),
    min(joined_df.Sales).alias('MIN(Sales)'),
    max(joined_df.Sales).alias('MAX(Sales)')
)
summary.show()

23/06/30 22:52:19 WARN TaskSetManager: Stage 17 contains a task of very large size (54752 KiB). The maximum recommended task size is 1000 KiB.


+------------------+-----------------+----------+----------+
|        AVG(Sales)|       STD(Sales)|MIN(Sales)|MAX(Sales)|
+------------------+-----------------+----------+----------+
|1722.4612253147923|967.3834016356338|        11|      3399|
+------------------+-----------------+----------+----------+



In [22]:
df_skew_salting = df_skew.withColumn('_salt_', round(rand() * 2))
df_small_salting = df_small.withColumn('_salt_', round(rand() * 2))

df_skew_salting.show()
df_small_salting.show()

+-------+---+-----+--------+------+
|OrderID|Qty|Sales|Discount|_salt_|
+-------+---+-----+--------+------+
|    201| 70|   75|       0|   0.0|
|    201| 49|   28|       0|   1.0|
|    201| 32|   79|       1|   1.0|
|    201| 42|   69|       1|   0.0|
|    201| 44|   39|       0|   1.0|
|    201| 51|   56|       1|   2.0|
|    201| 15|   47|       0|   2.0|
|    201| 95|   35|       1|   1.0|
|    201| 20|   73|       1|   2.0|
|    201|  5|   13|       0|   1.0|
|    201| 83|   16|       1|   0.0|
|    201| 65|   72|       1|   0.0|
|    201| 90|   46|       0|   1.0|
|    201|  9|   88|       0|   1.0|
|    201| 12|   18|       0|   0.0|
|    201| 85|   95|       0|   0.0|
|    201| 61|   91|       1|   0.0|
|    201| 69|   38|       1|   0.0|
|    201| 51|   30|       0|   1.0|
|    201| 43|   87|       1|   1.0|
+-------+---+-----+--------+------+
only showing top 20 rows

+---------+-------+----------+-----+------+
|ProductID|OrderID|   Product|Price|_salt_|
+---------+-------+---

23/06/30 23:05:04 WARN TaskSetManager: Stage 20 contains a task of very large size (54752 KiB). The maximum recommended task size is 1000 KiB.


In [23]:
# Repartition using salting
df_skew_salting = df_skew_salting.repartition(8, '_salt_')
df_small_salting = df_small_salting.repartition(8, '_salt_')

In [25]:
df_skew_salting = df_skew_salting.drop('_salt_')
df_small_salting = df_small_salting.drop('_salt_')

In [26]:
joined_df_salting = df_skew_salting.join(df_small_salting, on='OrderID', how='inner')

In [27]:
summary_salting = joined_df_salting.select(
    mean(joined_df.Sales).alias('AVG(Sales)'),
    stddev(joined_df.Sales).alias('STD(Sales)'),
    min(joined_df.Sales).alias('MIN(Sales)'),
    max(joined_df.Sales).alias('MAX(Sales)')
)
summary_salting.show()

23/06/30 23:09:02 WARN TaskSetManager: Stage 21 contains a task of very large size (54752 KiB). The maximum recommended task size is 1000 KiB.


+------------------+-----------------+----------+----------+
|        AVG(Sales)|       STD(Sales)|MIN(Sales)|MAX(Sales)|
+------------------+-----------------+----------+----------+
|1722.4612253147923|967.3834016355883|        11|      3399|
+------------------+-----------------+----------+----------+

